You want to explore the relationship between the individual LED currents and the total current. Feature engineering could potentially help with tasks like:

- Predicting individual LED currents based on the total current.
- Identifying abnormal current patterns for fault detection.


In [1]:
import math
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm, metrics, tree
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

# Set the backend to TkAgg for interactive plotting
plt.switch_backend('TkAgg')

/home/cas/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Read the data file
df = pd.read_csv("./labeled_dataset_1_small.csv", sep=',')

# Print the data for verification
print('\n\ndf.head(10):\n')
print(df.head(10))



df.head(10):

    current      R      G      B
0  0.036605  0.639  0.639  0.639
1  0.001844  0.396  0.000  0.396
2  0.012273  0.639  0.396  0.639
3  0.002571  0.639  0.000  0.396
4  0.012477  0.639  0.396  0.639
5  0.006664  0.396  0.396  0.639
6  0.012040  0.396  0.639  0.639
7  0.038608  0.639  0.639  0.639
8  0.001615  0.000  0.396  0.000
9  0.002370  0.639  0.000  0.396


In [3]:
# Generate a report for Pandas Dataframe analysis using the ydata profiling library
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_notebook_iframe()
# profile.to_file("report.html")

In [4]:
correlation_matrix = df.corr()
print(correlation_matrix)

          current         R         G         B
current  1.000000  0.360415  0.471377  0.415010
R        0.360415  1.000000  0.000523  0.028787
G        0.471377  0.000523  1.000000  0.030066
B        0.415010  0.028787  0.030066  1.000000


In [5]:
df_current = df['current'] 
df_engineering = (df['R'] * 0.360415) + (df['G'] * 0.471377) + (df['B'] * 0.415010)
df_engineering = pd.DataFrame({'current': df_current, 'engineered_data': df_engineering})
correlation_matrix = df_engineering.corr()
print(correlation_matrix)

                  current  engineered_data
current          1.000000         0.709605
engineered_data  0.709605         1.000000


In [6]:
x = df['current'].to_numpy()  # Select the first column
y = df[['R','G','B']].to_numpy()  # Select all columns except the first

# Assuming x is the total current and y is a 2D array with individual LED currents (rows, columns)
led_names = ['Red', 'Green', 'Blue']  # Adjust names as needed

# Create a figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(12, 4))  # Adjust figure size as desired

# Iterate through each LED and create a scatter plot
for i, led_name in enumerate(led_names):
  axes[i].scatter(x, y[:, i])  # i refers to the column index in y for each LED
  axes[i].set_title(f"{led_name} LED Current")
  axes[i].set_xlabel('Total Current')
  axes[i].set_ylabel(f'{led_name} Current')
  axes[i].grid(True)

# Adjust layout (optional)
fig.suptitle('Scatter Plots of Total Current vs. Individual LED Currents')
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
scaler = preprocessing.StandardScaler()
data = scaler.fit_transform(df_engineering)
print('\n')
print('scaler.fit_transform(x) - head(10):\n')
print(data[:10])



scaler.fit_transform(x) - head(10):

[[ 3.26316654  1.84615336]
 [-0.54226827 -0.74446667]
 [ 0.59943881  1.24011082]
 [-0.46268048 -0.28108636]
 [ 0.62177156  1.24011082]
 [-0.01460233  0.77673051]
 [ 0.5739313   1.38277305]
 [ 3.48244349  1.84615336]
 [-0.56733787 -1.38150533]
 [-0.48468481 -0.28108636]]


In [ ]:
# Put the data in variable X and the answers in variable y
x = data[:, 0]  # Select the first column
y = data[:, 1]
print("\n\nx - head(10):\n")
print(x[:10])
print("\ny - head(10):\n")
print(y[:10])



x - head(10):

[ 3.26316654 -0.54226827  0.59943881 -0.46268048  0.62177156 -0.01460233
  0.5739313   3.48244349 -0.56733787 -0.48468481]

y - head(10):

[ 1.84615336 -0.74446667  1.24011082 -0.28108636  1.24011082  0.77673051
  1.38277305  1.84615336 -1.38150533 -0.28108636]


In [ ]:
x = data[:, 0]  # Select the first column
y = data[:, 1]  # Select all columns except the first

# Assuming x and y are your data arrays
correlation_coefficient = np.corrcoef(x, y)[0, 1]
print("Correlation Coefficient:", correlation_coefficient)

Correlation Coefficient: 0.709604874025887


In [ ]:
x = data[:, 0]  # Select the first column
y = data[:, 1]  # Select all columns except the first
# x = x.reshape(-1, 1)

# Split the dataset into training (80%) and test (20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

def fitted_polynomial(x):
  """
  This function represents the fitted 10th-degree polynomial equation.
  """
  # Coefficients extracted from the equation string
  coeffs = fitted_coeffs

  # Evaluate the polynomial using coefficients and x
  return coeffs[0] + sum(coeff * x**i for i, coeff in enumerate(coeffs[1:]))

# Assuming x_test is a NumPy array
y_predicted = fitted_polynomial(x_test)  # Element-wise multiplication
y_predicted = (y_predicted * -1.0) + 2.5

# Plot the actual data (x vs y)
plt.scatter(x_test, y_test, label='Actual Data', color='blue')

# Plot the predicted values (x vs y_predicted)
plt.scatter(x_test, y_predicted, label='Predicted Data', color='red')

# Add labels and title
plt.xlabel('x')
plt.ylabel('y')
plt.title('Actual vs. Predicted Values')

# Add legend
plt.legend()

# Show the plot
plt.grid(True)  # Add grid for better visualization (optional)
plt.show()

In [ ]:
# Plot the actual data (x vs y)
plt.scatter(x_test, y_test, label='Actual Data', color='blue')

# Plot the predicted values (x vs y_predicted)
plt.scatter(x_test, y_predicted, label='Predicted Data', color='red')

# Add labels and title
plt.xlabel('x')
plt.ylabel('y')
plt.title('Actual vs. Predicted Values')

# Add legend
plt.legend()

# Show the plot
plt.grid(True)  # Add grid for better visualization (optional)
plt.show()

In [ ]:
from lmfit import Parameters, minimize

# Assuming data is a 2D array with x in the first column and y in the second
x = data[:, 0]  # Select the first column
y = data[:, 1]  # Select the second column

# Invert y values (assuming they represent positive values)
# y_inv = 1 / y
y_inv = y

params = Parameters()
params.add('a', value=1.0)  # Initial guess for parameter a
params.add('b', value=0.1)  # Initial guess for parameter b
params.add('c', value=0.0)  # Initial guess for parameter c (baseline)

# Define the objective function (difference between inverted y and model output)
def objective_function(params, x, y_inv):
  a = params['a']
  b = params['b']
  c = params['c']
  model_output = a * np.exp(-b * x) + c

  # Calculate squared errors for each data point
  squared_errors = (y_inv - model_output) ** 2

  # Return the total sum of squared errors
  return np.sum(squared_errors), 0, 0

# Fit the model to your data
result = minimize(objective_function, params, args=(x, y_inv))

# Access fitted parameters
a_fitted = result.params['a'].value
b_fitted = result.params['b'].value
c_fitted = result.params['c'].value

# Print the fitted function (consider reversing the transformation if applied)
print(f"Fitted function: y = {a_fitted} * exp(-{b_fitted} * x) + {c_fitted}")

# Optional: Calculate residuals (difference between inverted y and fitted model)
residuals = y_inv - (a_fitted * np.exp(-b_fitted * x) + c_fitted)
print(f"Residuals (sum of squared errors): {np.sum(residuals**2)}")


Fitted function: y = -0.31984337736363 * exp(-3.617298908434978 * x) + 1.292861663911042
Residuals (sum of squared errors): 111.90411344034275


In [ ]:
# from scipy.optimize import curve_fit

# # Assuming data is a 2D array with x in the first column and y in the second
# x = data[:, 0]  # Select the first column
# y = data[:, 1]  # Select the second column

# # Define cubic polynomial function
# def poly_func(x, a, b, c, d, e, f):
#   return a * x**5 + b * x**4 + c * x**3 + d * x**2 + e * x + f

# # Perform curve fitting
# popt, pcov = curve_fit(poly_func, x, y)

# # Access fitted coefficients
# a_fitted, b_fitted, c_fitted, d_fitted, e_fitted, f_fitted= popt

# # Construct the fitted polynomial equation
# fitted_equation = f"y = {a_fitted:.4f}x^5 + {b_fitted:.4f}x^4 + {c_fitted:.4f}x^3 + {d_fitted:.4f}x^2 + {e_fitted:.4f}x + {f_fitted:.4f}"

# # Print the fitted equation
# print(fitted_equation)

# # (Optional) Visualization using matplotlib
# import matplotlib.pyplot as plt

# plt.plot(x, y, 'o', label='Data')
# plt.plot(x, poly_func(x, *popt), label='Fitted Curve')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.title('Fitted Cubic Polynomial')
# plt.legend()
# plt.grid(True)
# plt.show()


In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Assuming x and y are your data arrays
x = data[:, 0]  # Select the first column
y = data[:, 1]  # Select the second column

print(x)
print()
print(y)
print()

# Define a function for a 10th-degree polynomial (modified to return an array)
def poly_func(x, *coeffs):
  """
  This function represents a polynomial of any degree.
  *coeffs: coefficients of the polynomial (a0, a1, ..., a10)
  """
  y_values = []
  for x_val in x:
    y = np.sum([coeff * x_val**i for i, coeff in enumerate(coeffs)])
    y_values.append(y)
  return np.array(y_values)

# Create a list of coefficients for a 10th-degree polynomial (adjust n_coeffs as needed)
n_coeffs = 11  # Number of coefficients for a 10th-degree polynomial (a0 to a10)
init_coeffs = np.ones(n_coeffs)  # Initial guess for coefficients (all ones)

# Perform curve fitting using least squares
popt, pcov = curve_fit(poly_func, x, y, p0=init_coeffs)

# Access fitted coefficients
fitted_coeffs = popt

# Print the fitted polynomial equation (symbolic representation)
print("Fitted polynomial equation (symbolic):")
for i, coeff in enumerate(fitted_coeffs):
  if i == 0:
    print(f"{coeff:.4f}", end="")  # Print constant term
  elif i == 1:
    print(f" + {coeff:.4f}x", end="")  # Print linear term
  else:
    print(f" + {coeff:.4f}x^{i}", end="")  # Print higher-degree terms

# (Optional) Print the fitted polynomial equation (numerical representation)
fitted_eqn = ""
for i, coeff in enumerate(fitted_coeffs):
  if i == 0:
    fitted_eqn += f"{coeff:.4f}"
  elif i == 1:
    if coeff > 0:
      fitted_eqn += f" + {coeff:.4f}*x"
    else:
      fitted_eqn += f" - {abs(coeff):.4f}*x"
  else:
    if coeff > 0:
      fitted_eqn += f" + {coeff:.4f}*x**{i}"
    else:
      fitted_eqn += f" - {abs(coeff):.4f}*x**{i}"
print(f"\nFitted polynomial equation (numerical): y = {fitted_eqn}")

# (Optional) Visualization using matplotlib
plt.plot(x, y, 'o', label='Data')
plt.plot(x, poly_func(x, *popt), 'o-', label='Fitted Curve')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Fitted 10th-Degree Polynomial')
plt.legend()
plt.show()


[ 3.26316654 -0.54226827  0.59943881 ... -0.47570792 -0.28456586
 -0.47844477]

[ 1.84615336 -0.74446667  1.24011082 ... -0.0203245   0.25248595
 -0.0203245 ]

Fitted polynomial equation (symbolic):
0.6242 + 0.8904x + 1.5136x^2 + 0.6657x^3 + -8.5757x^4 + 10.8173x^5 + -6.2347x^6 + 1.9698x^7 + -0.3524x^8 + 0.0336x^9 + -0.0013x^10
Fitted polynomial equation (numerical): y = 0.6242 + 0.8904*x + 1.5136*x**2 + 0.6657*x**3 - 8.5757*x**4 + 10.8173*x**5 - 6.2347*x**6 + 1.9698*x**7 - 0.3524*x**8 + 0.0336*x**9 - 0.0013*x**10


In [ ]:
x = data[:, 0]  # Select the first column
y = data[:, 1]  # Select all columns except the first
x = x.reshape(-1, 1)

# Split the dataset into training (80%) and test (20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Define and train the ML model
clf = LinearRegression()
# clf = LogisticRegression()
# clf = svm.SVC(probability=True)
# clf = tree.DecisionTreeClassifier()
# clf = MLPClassifier(random_state=1,max_iter=500)
clf.fit(x_train, y_train)
# tree.plot_tree(clf) 


#implement the Cross-Validation (CV) technique
cv = KFold(n_splits=5)
print(f"\nNumber of folds: {cv.get_n_splits(x_train)}\n")
print(cv)


# # Create and train the model
# model = LogisticRegression()
# model.fit(x_train, y_train)

# Make predictions on the test set

# y_predicted = clf.predict(x_test)

# Define the exponential expression (replace with your actual function if needed)
def exponential_expression(x):
  # return 0.0267*x**5 + -0.3416*x**4 + 1.4774*x**3 + -2.4925*x**2 + 1.5757*x + 0.9241
  return 0.6242 + 0.8904*x + 1.5136*x**2 + 0.6657*x**3 - 8.5757*x**4 + 10.8173*x**5 - 6.2347*x**6 + 1.9698*x**7 - 0.3524*x**8 + 0.0336*x**9 - 0.0013*x**10

# Assuming x_test is a NumPy array
y_predicted = exponential_expression(x_test)  # Element-wise multiplication

# Print the result (optional)
print(y_predicted)

# # Evaluate the model's performance (e.g., using Mean Squared Error (MSE))
# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(y_test, y_predicted)
# print("Mean Squared Error:", mse)


Number of folds: 5

KFold(n_splits=5, random_state=None, shuffle=False)
[[ 3.33099535e-01]
 [-3.21830461e-01]
 [ 1.00791428e+02]
 [-2.38554806e-01]
 [-6.53957771e-01]
 [-1.44722719e+00]
 [-1.74366152e+00]
 [ 2.80489255e-01]
 [ 1.74606260e-01]
 [-3.99489942e-01]
 [ 3.11381462e-01]
 [-8.13184440e-01]
 [-1.08484543e+00]
 [ 6.33484708e-01]
 [-9.60890064e-01]
 [ 3.90724191e-01]
 [-3.51003615e-01]
 [-1.23891688e+00]
 [-3.57792591e-01]
 [ 7.01428194e-01]
 [ 1.23459072e+00]
 [ 5.40466544e-01]
 [ 1.40202541e+00]
 [-8.20504630e-01]
 [ 3.32216845e-01]
 [ 3.26844934e-01]
 [ 2.19691459e-01]
 [-1.51415742e+00]
 [ 4.50651114e-01]
 [-8.82846094e-01]
 [ 5.98443375e-01]
 [ 4.31600298e-01]
 [ 6.80369030e-01]
 [-9.25326252e-01]
 [ 6.29080927e-01]
 [-1.31094861e+00]
 [-3.64627039e-01]
 [-1.18123275e+00]
 [ 6.93887393e-01]
 [-8.36482342e-01]
 [-1.30141420e+00]
 [ 1.19858644e+00]
 [ 2.46578503e-01]
 [ 5.51441587e-01]
 [-1.24662852e+00]
 [-8.26630865e-01]
 [-6.08841791e-01]
 [ 3.30170472e+01]
 [-6.94968470e-